In [1]:
computer = 'win'

if computer == 'mac':
    str_chromedriver = '../../../../chromedriver'
elif computer == 'win':
    str_chromedriver = '../../../../chromedriver.exe'
else:
    print('？？？')
    exit(0)

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import Proxy
import UA

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./part.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Info'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=' + Proxy.get_Proxy_Selenium())
option.add_argument('user-agent=' + UA.get_User_Agent())
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = =

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

# = = = = = = = = = = = = = = =

import time

# = = = = = = = = = = = = = = =

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                browser = Chrome(service=Service(str_chromedriver), options=option)

                browser.maximize_window()
                
                # = = = = = = = = = = = = = = =
                
                browser.get(url)

                time.sleep(3)
                
                # = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60)
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[contains(@aria-label, "More Information for")]')))
                list_tr = browser.find_elements(by=By.XPATH, value='//section[@aria-label="Components that come in this kit"]/descendant::tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-1") or contains(@class, "altrow-b-0"))]/tr')

                # = = = = = = = = = = = = = = =
                
                if len(list_tr) == 0:
                    break
                
                # = = = = = = = = = = = = = = =
                
                list_no = [_ + 1 for _ in range(len(list_tr))]
                list_quantity = []
                list_note = []
                list_manufacturer = []
                list_part_number = []
                list_info = []
                for _ in list_tr:
                    list_td = _.find_elements(by=By.XPATH, value='./td')
                    list_quantity.append(list_td[0].text)
                    list_note.append(list_td[1].text)
                    list_manufacturer.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-manufacturer")]').text)
                    list_part_number.append(list_td[2].find_element(by=By.XPATH, value='./descendant::span[contains(@class, "listing-final-partnumber")]').text)
                    list_info.append(list_td[2].find_element(by=By.XPATH, value='./descendant::a[contains(@class, "ra-btn") and contains(@class, "ra-btn-moreinfo")]').get_attribute('href'))

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Info': url,
                                        'Kit_序号': list_no,
                                        'Kit_Quantity': list_quantity,
                                        'Kit_Note': list_note,
                                        'Kit_Manufacturer': list_manufacturer,
                                        'Kit_Part_Number': list_part_number,
                                        'Kit_OE': '',
                                        'Kit_Pic': '',
                                        'Kit_Info': list_info,
                                        'Kit_Src': ''})
                
                browser.quit()
                
                status = 'ok'
                
                break
                
            except:
                browser.quit()
                time.sleep(3)
                continue
                
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Info': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./kit_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：835

https://www.rockauto.com/en/moreinfo.php?pk=12805185&cc=0&pt=11844  <->  [ok] - 剩余数量：820
https://www.rockauto.com/en/moreinfo.php?pk=12805205&cc=0&pt=11844  <->  [ok] - 剩余数量：819
https://www.rockauto.com/en/moreinfo.php?pk=12805209&cc=0&pt=11844  <->  [ok] - 剩余数量：818
https://www.rockauto.com/en/moreinfo.php?pk=12805193&cc=0&pt=11844  <->  [ok] - 剩余数量：817
https://www.rockauto.com/en/moreinfo.php?pk=12805161&cc=0&pt=11844  <->  [ok] - 剩余数量：816
https://www.rockauto.com/en/moreinfo.php?pk=12805169&cc=0&pt=11844  <->  [ok] - 剩余数量：815
https://www.rockauto.com/en/moreinfo.php?pk=12805153&cc=0&pt=11844  <->  [ok] - 剩余数量：814
https://www.rockauto.com/en/moreinfo.php?pk=12805189&cc=0&pt=11844  <->  [ok] - 剩余数量：813
https://www.rockauto.com/en/moreinfo.php?pk=12805173&cc=0&pt=11844  <->  [ok] - 剩余数量：812
https://www.rockauto.com/en/moreinfo.php?pk=12805201&cc=0&pt=11844  <->  [ok] - 剩余数量：811
https://www.rockauto.com/en/moreinfo.php?pk=12805197&cc=0&pt=11844  <->  [ok] - 剩余数量：810
https://www.

https://www.rockauto.com/en/moreinfo.php?pk=12805573&cc=0&pt=11844  <->  [ok] - 剩余数量：728
https://www.rockauto.com/en/moreinfo.php?pk=12805261&cc=0&pt=11844  <->  [ok] - 剩余数量：727
https://www.rockauto.com/en/moreinfo.php?pk=12805561&cc=0&pt=11844  <->  [ok] - 剩余数量：726
https://www.rockauto.com/en/moreinfo.php?pk=12805593&cc=0&pt=11844  <->  [ok] - 剩余数量：725
https://www.rockauto.com/en/moreinfo.php?pk=12805569&cc=0&pt=11844  <->  [ok] - 剩余数量：724
https://www.rockauto.com/en/moreinfo.php?pk=12805521&cc=0&pt=11844  <->  [ok] - 剩余数量：723
https://www.rockauto.com/en/moreinfo.php?pk=12805565&cc=0&pt=11844  <->  [ok] - 剩余数量：722
https://www.rockauto.com/en/moreinfo.php?pk=12805621&cc=0&pt=11844  <->  [ok] - 剩余数量：721
https://www.rockauto.com/en/moreinfo.php?pk=12805581&cc=0&pt=11844  <->  [ok] - 剩余数量：720
https://www.rockauto.com/en/moreinfo.php?pk=12805609&cc=0&pt=11844  <->  [ok] - 剩余数量：719
https://www.rockauto.com/en/moreinfo.php?pk=12805577&cc=0&pt=11844  <->  [ok] - 剩余数量：718
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13017013&cc=0&pt=11844  <->  [ok] - 剩余数量：635
https://www.rockauto.com/en/moreinfo.php?pk=13017017&cc=0&pt=11844  <->  [ok] - 剩余数量：634
https://www.rockauto.com/en/moreinfo.php?pk=13016977&cc=0&pt=11844  <->  [ok] - 剩余数量：633
https://www.rockauto.com/en/moreinfo.php?pk=13017037&cc=0&pt=11844  <->  [ok] - 剩余数量：632
https://www.rockauto.com/en/moreinfo.php?pk=13016889&cc=0&pt=11844  <->  [ok] - 剩余数量：631
https://www.rockauto.com/en/moreinfo.php?pk=13017001&cc=0&pt=11844  <->  [ok] - 剩余数量：630
https://www.rockauto.com/en/moreinfo.php?pk=13017025&cc=0&pt=11844  <->  [ok] - 剩余数量：629
https://www.rockauto.com/en/moreinfo.php?pk=13017029&cc=0&pt=11844  <->  [ok] - 剩余数量：628
https://www.rockauto.com/en/moreinfo.php?pk=13017053&cc=0&pt=11844  <->  [ok] - 剩余数量：627
https://www.rockauto.com/en/moreinfo.php?pk=13017049&cc=0&pt=11844  <->  [ok] - 剩余数量：626
https://www.rockauto.com/en/moreinfo.php?pk=13017041&cc=0&pt=11844  <->  [ok] - 剩余数量：625
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13017377&cc=0&pt=11844  <->  [ok] - 剩余数量：541
https://www.rockauto.com/en/moreinfo.php?pk=13017437&cc=0&pt=11844  <->  [ok] - 剩余数量：540
https://www.rockauto.com/en/moreinfo.php?pk=13017445&cc=0&pt=11844  <->  [ok] - 剩余数量：539
https://www.rockauto.com/en/moreinfo.php?pk=13017433&cc=0&pt=11844  <->  [ok] - 剩余数量：538
https://www.rockauto.com/en/moreinfo.php?pk=13017281&cc=0&pt=11844  <->  [ok] - 剩余数量：537
https://www.rockauto.com/en/moreinfo.php?pk=13017449&cc=0&pt=11844  <->  [ok] - 剩余数量：536
https://www.rockauto.com/en/moreinfo.php?pk=13017461&cc=0&pt=11844  <->  [ok] - 剩余数量：535
https://www.rockauto.com/en/moreinfo.php?pk=13017465&cc=0&pt=11844  <->  [ok] - 剩余数量：534
https://www.rockauto.com/en/moreinfo.php?pk=13017453&cc=0&pt=11844  <->  [ok] - 剩余数量：533
https://www.rockauto.com/en/moreinfo.php?pk=13017477&cc=0&pt=11844  <->  [ok] - 剩余数量：532
https://www.rockauto.com/en/moreinfo.php?pk=13017473&cc=0&pt=11844  <->  [ok] - 剩余数量：531
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13017857&cc=0&pt=11844  <->  [ok] - 剩余数量：448
https://www.rockauto.com/en/moreinfo.php?pk=13017865&cc=0&pt=11844  <->  [ok] - 剩余数量：447
https://www.rockauto.com/en/moreinfo.php?pk=13017861&cc=0&pt=11844  <->  [ok] - 剩余数量：446
https://www.rockauto.com/en/moreinfo.php?pk=13017761&cc=0&pt=11844  <->  [ok] - 剩余数量：445
https://www.rockauto.com/en/moreinfo.php?pk=13017869&cc=0&pt=11844  <->  [ok] - 剩余数量：444
https://www.rockauto.com/en/moreinfo.php?pk=13017881&cc=0&pt=11844  <->  [ok] - 剩余数量：443
https://www.rockauto.com/en/moreinfo.php?pk=13017885&cc=0&pt=11844  <->  [ok] - 剩余数量：442
https://www.rockauto.com/en/moreinfo.php?pk=13017889&cc=0&pt=11844  <->  [ok] - 剩余数量：441
https://www.rockauto.com/en/moreinfo.php?pk=13017821&cc=0&pt=11844  <->  [ok] - 剩余数量：440
https://www.rockauto.com/en/moreinfo.php?pk=13017825&cc=0&pt=11844  <->  [ok] - 剩余数量：439
https://www.rockauto.com/en/moreinfo.php?pk=13017897&cc=0&pt=11844  <->  [ok] - 剩余数量：438
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13400897&cc=0&pt=11844  <->  [ok] - 剩余数量：355
https://www.rockauto.com/en/moreinfo.php?pk=13400917&cc=0&pt=11844  <->  [ok] - 剩余数量：354
https://www.rockauto.com/en/moreinfo.php?pk=13400921&cc=0&pt=11844  <->  [ok] - 剩余数量：353
https://www.rockauto.com/en/moreinfo.php?pk=13400913&cc=0&pt=11844  <->  [ok] - 剩余数量：352
https://www.rockauto.com/en/moreinfo.php?pk=13400929&cc=0&pt=11844  <->  [ok] - 剩余数量：351
https://www.rockauto.com/en/moreinfo.php?pk=13400869&cc=0&pt=11844  <->  [ok] - 剩余数量：350
https://www.rockauto.com/en/moreinfo.php?pk=13400905&cc=0&pt=11844  <->  [ok] - 剩余数量：349
https://www.rockauto.com/en/moreinfo.php?pk=13400861&cc=0&pt=11844  <->  [ok] - 剩余数量：348
https://www.rockauto.com/en/moreinfo.php?pk=13400925&cc=0&pt=11844  <->  [ok] - 剩余数量：347
https://www.rockauto.com/en/moreinfo.php?pk=13400937&cc=0&pt=11844  <->  [ok] - 剩余数量：346
https://www.rockauto.com/en/moreinfo.php?pk=13400945&cc=0&pt=11844  <->  [ok] - 剩余数量：345
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13401333&cc=0&pt=11844  <->  [ok] - 剩余数量：262
https://www.rockauto.com/en/moreinfo.php?pk=13401313&cc=0&pt=11844  <->  [ok] - 剩余数量：261
https://www.rockauto.com/en/moreinfo.php?pk=13401345&cc=0&pt=11844  <->  [ok] - 剩余数量：260
https://www.rockauto.com/en/moreinfo.php?pk=13401341&cc=0&pt=11844  <->  [ok] - 剩余数量：259
https://www.rockauto.com/en/moreinfo.php?pk=13401357&cc=0&pt=11844  <->  [ok] - 剩余数量：258
https://www.rockauto.com/en/moreinfo.php?pk=13401349&cc=0&pt=11844  <->  [ok] - 剩余数量：257
https://www.rockauto.com/en/moreinfo.php?pk=13401365&cc=0&pt=11844  <->  [ok] - 剩余数量：256
https://www.rockauto.com/en/moreinfo.php?pk=13401373&cc=0&pt=11844  <->  [ok] - 剩余数量：255
https://www.rockauto.com/en/moreinfo.php?pk=13401377&cc=0&pt=11844  <->  [ok] - 剩余数量：254
https://www.rockauto.com/en/moreinfo.php?pk=13401369&cc=0&pt=11844  <->  [ok] - 剩余数量：253
https://www.rockauto.com/en/moreinfo.php?pk=13401389&cc=0&pt=11844  <->  [ok] - 剩余数量：252
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13401745&cc=0&pt=11844  <->  [ok] - 剩余数量：168
https://www.rockauto.com/en/moreinfo.php?pk=13401741&cc=0&pt=11844  <->  [ok] - 剩余数量：167
https://www.rockauto.com/en/moreinfo.php?pk=13401757&cc=0&pt=11844  <->  [ok] - 剩余数量：166
https://www.rockauto.com/en/moreinfo.php?pk=13401761&cc=0&pt=11844  <->  [ok] - 剩余数量：165
https://www.rockauto.com/en/moreinfo.php?pk=13401765&cc=0&pt=11844  <->  [ok] - 剩余数量：164
https://www.rockauto.com/en/moreinfo.php?pk=13401773&cc=0&pt=11844  <->  [ok] - 剩余数量：163
https://www.rockauto.com/en/moreinfo.php?pk=13401769&cc=0&pt=11844  <->  [ok] - 剩余数量：162
https://www.rockauto.com/en/moreinfo.php?pk=13401781&cc=0&pt=11844  <->  [ok] - 剩余数量：161
https://www.rockauto.com/en/moreinfo.php?pk=13401789&cc=0&pt=11844  <->  [ok] - 剩余数量：160
https://www.rockauto.com/en/moreinfo.php?pk=13401777&cc=0&pt=11844  <->  [ok] - 剩余数量：159
https://www.rockauto.com/en/moreinfo.php?pk=13401801&cc=0&pt=11844  <->  [ok] - 剩余数量：158
https://www.rockauto.

https://www.rockauto.com/en/moreinfo.php?pk=13825573&cc=0&pt=11844  <->  [ok] - 剩余数量：75
https://www.rockauto.com/en/moreinfo.php?pk=13825561&cc=0&pt=11844  <->  [ok] - 剩余数量：74
https://www.rockauto.com/en/moreinfo.php?pk=14832889&cc=0&pt=11844  <->  [ok] - 剩余数量：73
https://www.rockauto.com/en/moreinfo.php?pk=14832901&cc=0&pt=11844  <->  [ok] - 剩余数量：72
https://www.rockauto.com/en/moreinfo.php?pk=14832917&cc=0&pt=11844  <->  [ok] - 剩余数量：71
https://www.rockauto.com/en/moreinfo.php?pk=14832893&cc=0&pt=11844  <->  [ok] - 剩余数量：70
https://www.rockauto.com/en/moreinfo.php?pk=14832909&cc=0&pt=11844  <->  [ok] - 剩余数量：69
https://www.rockauto.com/en/moreinfo.php?pk=14832925&cc=0&pt=11844  <->  [ok] - 剩余数量：68
https://www.rockauto.com/en/moreinfo.php?pk=14832913&cc=0&pt=11844  <->  [ok] - 剩余数量：67
https://www.rockauto.com/en/moreinfo.php?pk=14832921&cc=0&pt=11844  <->  [ok] - 剩余数量：66
https://www.rockauto.com/en/moreinfo.php?pk=14832933&cc=0&pt=11844  <->  [ok] - 剩余数量：65
https://www.rockauto.com/en/more

D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
